In [42]:
import yfinance as yf
import matplotlib as plt
from indicators import *
import os
import importlib
importlib.reload(indicators)
print(dir(indicators))

['BollingerBands', 'EMA', 'MA', 'MACD', 'RSI', 'STD', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'pandas', 'pd']


In [43]:
start_date = '1950-01-01'
end_date = '2025-01-01'

In [89]:
indices = {
    "SP500": "^GSPC",
    "Nasdaq100": "^NDX",
    "DowJones": "^DJI",
    "Russell2000": "^RUT",
    "FTSE100": "^FTSE",
    "DAX": "^GDAXI",
    "CAC40": "^FCHI",
    "Nikkei225": "^N225",
    "HangSeng": "^HSI",
    "EuroStoxx50": "^STOXX50E"
}

In [85]:
def extract_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date, interval="1d")

    data = data[data["Volume"] > 0]
    data["V_t"] = (data["Close"].diff() / data["Close"].shift()) * 100
    data["MA5"] = MA(data["Close"], 5)
    data["MA15"] = MA(data["Close"], 15)
    data["EMA5"] = EMA(data["Close"], 5)
    data["EMA15"] = EMA(data["Close"], 15)
    data["RSI14"] = RSI(data["Close"], 14)
    data["MACD"], data["Signal"] = MACD(data["Close"], 12, 26)
    data["STD10"] = STD(data["Close"], 10)
    data["STD20"] = STD(data["Close"], 20)
    data["BollingerBandUpper"], data["BollingerBandLower"] = BollingerBands(data["Close"], 20, 2)
    data = data.dropna()

    data = data.drop("Adj Close", axis=1)
    if isinstance(data.columns, pd.MultiIndex):
        # On garde seulement le premier niveau de header
        data.columns = data.columns.get_level_values(0)

    # On transforme la date en colonne
    data = data.reset_index()
    data["Date"] = pd.to_datetime(data["Date"]).dt.date

    return data

In [91]:
base_list = []

In [92]:
for name, ticker in indices.items():
    base_dir = os.path.join(os.getcwd(), "base")

    data = extract_data(ticker, start_date, end_date)

    data.insert(0, 'Indice', name)

    base_list.append(data)

    
    if data is not None:
        file_path = os.path.join(base_dir, f"{name}.csv")
        data.to_csv(file_path)
        print(f"✅ Fichier enregistré : {file_path}")

    BaseTotal = pd.concat(base_list)

    file_path = os.path.join(base_dir, "BaseTotal.csv")
    BaseTotal.to_csv(file_path)

[*********************100%***********************]  1 of 1 completed


✅ Fichier enregistré : /Users/romaindesmeulemester/ProjetMachineLearning/ProjetMachineLearning/base/SP500.csv


[*********************100%***********************]  1 of 1 completed


✅ Fichier enregistré : /Users/romaindesmeulemester/ProjetMachineLearning/ProjetMachineLearning/base/Nasdaq100.csv


[*********************100%***********************]  1 of 1 completed


...

✅ Fichier enregistré : /Users/romaindesmeulemester/ProjetMachineLearning/ProjetMachineLearning/base/Nikkei225.csv


[*********************100%***********************]  1 of 1 completed


✅ Fichier enregistré : /Users/romaindesmeulemester/ProjetMachineLearning/ProjetMachineLearning/base/HangSeng.csv


[*********************100%***********************]  1 of 1 completed


✅ Fichier enregistré : /Users/romaindesmeulemester/ProjetMachineLearning/ProjetMachineLearning/base/EuroStoxx50.csv


In [101]:
BaseTotal = BaseTotal.reset_index(drop=True)

BaseTrain = BaseTotal.sample(frac=0.8, random_state=0)

BaseTest = BaseTotal.drop(BaseTrain.index)



Price,Indice,Date,Close,High,Low,Open,Volume,V_t,MA5,MA15,EMA5,EMA15,RSI14,MACD,Signal,STD10,STD20,BollingerBandUpper,BollingerBandLower,ID
2,SP500,1950-02-01,17.049999,17.049999,17.049999,17.049999,1810000.0,0.000000,16.934000,16.854667,16.972734,16.893426,66.666594,0.057229,0.041967,0.116923,0.130718,17.157935,16.635065,3
5,SP500,1950-02-06,17.320000,17.320000,17.320000,17.320000,1490000.0,0.173504,17.188000,16.967333,17.197106,17.022334,76.744114,0.116766,0.070797,0.220124,0.190868,17.320736,16.557264,6
10,SP500,1950-02-14,17.059999,17.059999,17.059999,17.059999,2210000.0,-1.044085,17.204000,17.075333,17.176656,17.106440,58.928483,0.110911,0.104715,0.103731,0.196717,17.419934,16.633066,11
13,SP500,1950-02-17,17.150000,17.150000,17.150000,17.150000,1940000.0,0.941730,17.100000,17.133333,17.109009,17.094705,64.601728,0.077061,0.094073,0.112749,0.185014,17.427528,16.687471,14
19,SP500,1950-02-28,17.219999,17.219999,17.219999,17.219999,1310000.0,-0.347230,17.232000,17.193333,17.227320,17.170290,44.318204,0.082276,0.084670,0.097502,0.102398,17.381795,16.972204,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74812,EuroStoxx50,2023-11-29,4370.529785,4382.149902,4341.299805,4341.299805,30478700.0,0.517702,4361.275977,4316.128678,4357.289174,4309.924103,73.527475,58.700953,52.031811,20.382278,81.705911,4442.247857,4115.424213,2353
74815,EuroStoxx50,2023-12-04,4414.950195,4426.549805,4405.330078,4414.299805,25291700.0,-0.080560,4386.895996,4353.274023,4393.844615,4341.871811,77.300843,62.118744,56.403455,28.099899,77.109942,4468.705920,4160.266150,2356
74819,EuroStoxx50,2023-12-08,4523.310059,4531.540039,4477.040039,4477.040039,26886000.0,1.107344,4469.612012,4398.783333,4473.827446,4401.795869,82.750457,75.157428,64.970202,59.589437,70.754816,4514.730141,4231.710875,2360
74820,EuroStoxx50,2023-12-11,4540.189941,4544.129883,4519.620117,4522.790039,24134800.0,0.373175,4494.659961,4411.968652,4495.948278,4419.095128,86.859123,79.101954,67.796552,64.835569,71.951961,4532.524429,4244.716587,2361
